# Imports

In [19]:
import re
import string
import numpy as np
import pandas as pd
from pprint import pprint

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.feature_extraction import text

In [34]:
import spacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/isanavarro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/isanavarro/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/isanavarro/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


# Creación de Stop Words

In [5]:
nltk.download("stopwords")
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','a','about', 'above', 'across'])
st1= ['after', 'afterwards','again','against', 'all', 'almost','alone','along','already','also','although','always','am','among','amongst','amoungst','amount','an','and','another','any',
      'anyhow','anyone','anything','anyway','anywhere','are','around','as','at','back','be','became','because','become','becomes','becoming','been','before','beforehand','behind','being',
      'below','beside','besides','between','beyond','bill','both','bottom','but','by','call','can','cannot','cant','co','con','could','couldnt','cry','de','describe','detail','do','done',
      'down','due','during','each','eg','eight','either','eleven','else','elsewhere','empty','enough','etc','even','ever','every','everyone','everything','everywhere','except','few','fifteen',
      'fifty','fill','find','fire','first','five','for','former','formerly','forty','found','four','from','front','full','further','get','give','go','had','has','hasnt','have','he','hence',
      'her','here','hereafter','hereby','herein','hereupon','hers','herself','him','himself','his','how','however','hundred','i','ie','if','in','inc','indeed','interest','into','is','it',
      'its','itself','keep','last','latter','latterly','least','less','ltd','made','many','may','me','meanwhile','might','mill','mine','more','moreover','most','mostly','move','much','must',
      'my','myself','name','namely','neither','never','nevertheless','next','nine','no','nobody','none','noone','nor','not','nothing','now','nowhere','of','off','often','on','once','one',
      'only','onto','or','other','others','otherwise','our','ours','ourselves','out','over','own','part','per','perhaps','please','put','rather','re','same','see','seem','seemed','seeming',
      'seems','serious','several','she','should','show','side','since','sincere','six','sixty','so','some','somehow','someone','something','sometime','sometimes','somewhere','still','such',
      'system','take','ten','than','that','the','their','them','themselves','then','thence','there','thereafter','thereby','therefore','therein','thereupon','these','they','thick','thin',
      'third','this','those','though','three','through','throughout','thru','thus','to','together','too','top','toward','towards','twelve','twenty','two','un','under','until','up','upon',
      'us','very','via','was','we','well','were','what','whatever','when','whence','whenever','where','whereafter','whereas','whereby','wherein','whereupon','wherever','whether','which',
      'while','whither','who','whoever','whole','whom','whose','why','will','with','within','without','would','yet','you','your','yours','yourself','yourselves']
stop_words.extend(st1)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/isanavarro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Lectura de Datos

In [22]:
pd.set_option('max_colwidth', 150)
df = pd.read_csv('JoeBiden_Filtered_TW.csv', engine='python')
df.head()

,Unnamed: 0,Filtered
0,0,"Through of organizing, negotiating, picketing, and protesting, labor unions secured vital workplace protections that union and non - union workers..."
1,1,Not only do Republicans oppose making big corporations pay their fair share— they want middle class families and small business owners to pay more...
2,2,"A stronger economy built from the bottom up and the middle out puts greater power in workers ' hands to improve their lives, provide for their fam..."
3,3,Democrats are working tirelessly to open doors for more outstanding entrepreneurs. \n\n But the Republican plan led by Senator would tax half of o...
4,4,"American workers are beginning to rebuild our nation 's roads, bridges, ports, and waterways. \n\n The Bipartisan Infrastructure Law will create n..."


In [26]:
df = df.reset_index()
df = df[['Filtered']]
df.head()

,Filtered
0,"Through of organizing, negotiating, picketing, and protesting, labor unions secured vital workplace protections that union and non - union workers..."
1,Not only do Republicans oppose making big corporations pay their fair share— they want middle class families and small business owners to pay more...
2,"A stronger economy built from the bottom up and the middle out puts greater power in workers ' hands to improve their lives, provide for their fam..."
3,Democrats are working tirelessly to open doors for more outstanding entrepreneurs. \n\n But the Republican plan led by Senator would tax half of o...
4,"American workers are beginning to rebuild our nation 's roads, bridges, ports, and waterways. \n\n The Bipartisan Infrastructure Law will create n..."


# Limpieza de datos

In [27]:
def clean_text_round1(text):
    text = text.lower()
    text = re.sub('\[.*?\]', ' ', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub('�', ' ', text)
    return text
round1 = lambda x: clean_text_round1(x)
df["Filtered"] = df["Filtered"].apply(round1)
df.head()

,Filtered
0,through of organizing negotiating picketing and protesting labor unions secured vital workplace protections that union and non union workers...
1,not only do republicans oppose making big corporations pay their fair share— they want middle class families and small business owners to pay more...
2,a stronger economy built from the bottom up and the middle out puts greater power in workers hands to improve their lives provide for their fam...
3,democrats are working tirelessly to open doors for more outstanding entrepreneurs \n\n but the republican plan led by senator would tax half of o...
4,american workers are beginning to rebuild our nation s roads bridges ports and waterways \n\n the bipartisan infrastructure law will create n...


# Función de extracción de sustantivos y lematización

In [35]:
def nouns(text):
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    wordnet_lemmatizer = WordNetLemmatizer()
    all_nouns = [wordnet_lemmatizer.lemmatize(word) \
    for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

data_nouns = pd.DataFrame(df.Filtered.apply(nouns))
data_nouns.head()

,Filtered
0,picketing labor union workplace protection union union worker union workplace protection
1,republican corporation share— class family business owner tax
2,economy bottom middle power worker hand life family quality job
3,democrat door entrepreneur plan senator tax half business owner year average
4,worker nation s road bridge port infrastructure law job community country infrastructure job people background


In [39]:
stop_noun = ["america", 'today', 'thing']
stop_words_noun_agg = text.ENGLISH_STOP_WORDS.union(stop_noun)
tv_noun = TfidfVectorizer(stop_words=stop_words_noun_agg, ngram_range = (1,1), max_df = .8, min_df = .01)
data_tv_noun = tv_noun.fit_transform(data_nouns.Filtered)
data_dtm_noun = pd.DataFrame(data_tv_noun.toarray(), columns=tv_noun.get_feature_names())
data_dtm_noun.index = df.index
data_dtm_noun.head()

,access,act,action,administration,advancement,agenda,airport,ally,american,arm,...,voting,wage,water,way,weather,woman,work,worker,world,year
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.221521,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.458764,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.336474
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.308001,0.0,0.000000


# Función para desplegar Topics

In [44]:
def display_topics(model, feature_names, num_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

# Aplicación de modelo NFM

In [57]:
nmf_model1 = NMF(2)
doc_topic = nmf_model1.fit_transform(data_dtm_noun)
nmf_model2 = NMF(8)
doc_topic = nmf_model2.fit_transform(data_dtm_noun)

/Users/isanavarro/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)
/Users/isanavarro/opt/anaconda3/lib/python3.7/site-packages/sklearn/decomposition/_nmf.py:315: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  "'nndsvda' in 1.1 (renaming of 0.26)."), FutureWarning)


# Visualización de resultados

In [61]:
print('Resultados para 2 temas------------------------------------------------------------------------')
display_topics(nmf_model1, tv_noun.get_feature_names(), 5)
print('')
print('Resultados para 8 temas------------------------------------------------------------------------')
display_topics(nmf_model, tv_noun.get_feature_names(), 10)

Resultados para 2 temas------------------------------------------------------------------------

Topic  0
job, infrastructure, law, country, economy

Topic  1
family, cost, build, act, american

Resultados para 8 temas------------------------------------------------------------------------

Topic  0
infrastructure, law, bridge, country, investment, water, road, internet, access, thanks

Topic  1
cost, build, act, insulin, family, drug, care, prescription, month, health

Topic  2
job, economy, history, growth, record, manufacturing, work, country, progress, innovation

Topic  3
booster, shot, omicron, variant, folk, vaccine, way, covid, protection, child

Topic  4
people, country, nation, challenge, world, vote, time, state, party, court

Topic  5
plan, tax, rescue, year, thanks, penny, deficit, administration, cut, growth

Topic  6
american, family, mask, work, million, unemployment, insurance, test, care, state

Topic  7
price, gas, family, hike, oil, petroleum, release, reserve, barr